1. 필수 라이브러리


In [ ]:
import sklearn as sk
import pandas as pd

In [ ]:
import sklearn.preprocessing as sk_pre
import sklearn.metrics as sk_met
import sklearn.model_selection as sk_mod

import seaborn as sns
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2. 데이터 불러오기 및 저장

In [ ]:
df = pd.read_csv("파일경로")
df_excel = pd.read_excel("파일 경로")

df.to_csv("저장할 파일 경로", index = False)
df.to_json("저장할 파일 경로", orient="records")
df.to_pickle("저장할 파일 경로")

3. 데이터 병합

In [ ]:
df1 = df[["model_year", "mpg", "cylinders"]] # 해당 컬럼만 포함
df2 = df[["model_year", "weight"]]

df_merge = pd.merge(df1, df2, on = "model_year", how="inner") # on: 해당 컬럼을 기준으로, how: 이렇게 조인해라 → inner, outer
df_concate = pd.concate([df1, df2], axis=0) # axis: 0→행(위, 아래로 조인), 1→열(좌, 우로 조인)

4. 데이터 분포 시각화

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.countplot(data=df, x="origin") # 해당 컬럼 분포 시각화
sns.histplot(data=df, x="mpg", bins=20, kde=True)
sns.boxplot(data=df, x="weight")
sns.violinplot(data=df, x="horsepower")
plt.title("Origin Distribution")
plt.show()

In [ ]:
# 해당 컬럼의 빈도를 막대그래프로 표현
plt.figure(figsize=(8,5))
df["model_year"].value_counts().sort_index().plot(kind="bar")

plt.title("Model Year Distribution")
plt.xlabel("Model Year")
plt.ylabel("Count")
plt.show()

5. 변수 간 관계 분석

In [ ]:
df.mode() # 최빈값
df.std() # 표준편차
df.var() # 분산
df.corr() # 상관계수

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.joinplot(data=df, x="weight", y="mpg", kind="scatter")

plt.show()

6. 이상치 제거

In [ ]:
# 결측치 확인
print(df.isnull().sum())

In [ ]:
df_temp = df[df["mpg"] <= 50]

Q1 = df["horsepower"].quantile(0.25) # 1사분위
Q3 = df["horsepower"].quantile(0.75) # 2사분위

# IQR: 데이터의 중간 50% 범위를 기준으로 이상치를 탐지하는 방법
IQR = Q3-Q1
df_temp = df[(df["horsepower"] >= Q1 - 1.5 * IQR) & (df["horsepower"] <= Q3 + 1.5*IQR)]
plt.show()

In [ ]:
from scipy import stats

# Z-score을 활용하여 이상치 제거
df_temp = df[(abs(stats.zscore(df["acceleration"])) < 3)]

7. 결측치 제거

In [ ]:
df.dropna() # 모든 컬럼을 대상으로 결측치가 포함된 행 제거
df.dropna(subset=["horsepower"]) # 특정 행
df.dropna(axis=1) # 결측치가 포함된 열

df.fillna(0) # 결측치 대체
df["horsepower"].fillna(df["horsepower"].mean())

8. 불필요한 컬럼 삭제

In [ ]:
df.drop(columns = ["name", "origin"])
df.drop(index = 2) # 특정 행 삭제
df.drop_duplicates(keep = "fisrt") # 중복된 행 삭제, 첫 번째 중복행을 유지 ("fisrt", "last", "False")
df.drop_duplicates(subset=["열 이름"]) # 특정 열을 기준으로 중복 여부 판단

df.select_dtypes(exclue=["object"]) # 모든 문자열 타입 컬럼 삭제
df.select_dtypes(exclue=["number"]) # 모든 숫자열 타입 컬럼 삭제

df.iloc[:,1:]
df.iloc[:,:5]
df.loc[:,["mpg", "weight", "horsepower"]]

In [ ]:
df.grouby(["그룹기준 열"])

df.pivot_table(index="성별", columns = "흉통유형", values = "최대심박수", aggfunc = "mean", fill_value=0)

9. 인코딩

In [ ]:
# 원핫 인코딩
pd.get_dummies(df)
pd.get_dummies(df, columns = ["origin"])
pd.get_dummies(df, columns = ["origin"], prefix = "origin") # 열 이름 origin_1

In [ ]:
# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["origin"] = le.fit_transform(df["origin"]) # 컬럼이 1,2,3으로 변환

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder()
df["origin"] = oe.fit_transform(df["origin"]) # 컬럼이 숫자값 1,2,3으로 변환

### 10. 데이터셋 분리 및 스케일링

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns = ["mpg"])
y = df["mpg"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 42) # stratify=y : 타켓 변수의 불균형이 심할 때 범주를 균형있게 분할해줌.

In [ ]:
# 데이터를 지정한 구간(bins)에 따라 나누고, 각 구간에 이름을 붙임
pd.cut(df["Age"], bins=[0,20,40,60,80], labels =["10대","20대","30대","40대 이상"])

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

11. 머신러닝 모델 학습

In [ ]:
# 트리기반 모델은 스케일링이 필요하지 않음.
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegreesor

# 선형 모델, 거리 기반은 스케일링이 필요함.
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [ ]:
# 모델 정의
dt_model = DecisionTreeRegressor(max_depth = 5, random_state = 1202)
lr_model = LinearRegression()

In [ ]:
# 모델 학습
dt_model.fit(X_train, y_train)

### 12. 모델 성능 평가

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
# 예측값 계산
y_pred = dt_model.predict(X_valid)

In [ ]:
# MAE 계산
dt_mae = mean_absolute_error(y_valid, y_pred)

13. 딥러닝 모델 학습
- TensorFlow를 활용하여 딥러닝 모델을 설계하세요.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.laeyers import Dense, Dropout

In [ ]:
# 모델 정의
model = Sequential([Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
                    Dropout(0.2),
                    Dense(32, activation = "relu"),
                    Dense(1)
                    ])

In [ ]:
model = Sequential()
model.add(Dense(64, activation = "tanh", input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation="tanh"))
model.add(Dense(1))

In [ ]:
model.compile(optimizer = "adam", loss="mse")
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=30, batch_size=16)

In [ ]:
from tensorflow.keras.optimizers import SGD

model.compile(optimizer = SGD(learning_rate = 0.01), loss ="mse")
model.fit(X_trian, y_train, validation_data=(X_valid, y_valid), epochs=30, batch_size=16)

In [ ]:
y_pred_svr = svr_model.predict(X_valid_scaled)

svr_mae = mean_absolute_error(y_valid, y_pred_svr)

In [ ]:
# 모델 저장 및 불러오기
model.save("saved_model.h5")

loaded_model = tf.keras.models.load_model("saved_model.h5")

In [ ]:
# 체크포인트 설정
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.h5", monitor="val_loss", save_best_only = True, mode="min")

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=30, batch_size=16, callbacks=[checkpoint])

In [ ]:
# 모델 학습 과정에서 저장된 손실값 가져오기
history = model.history.history

print("Final Training MSE:", history["loss"][-1])
print("Final Validation MSE:", history["val_loss"][-1])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("best_model.h5", mointor="val_loss", save_best_only=True, mode="min")
early_stopping = EarlyStopping(monitor = "val_loss", patience=5, restore_best_weights=True)

model.fit(X_train, y_train, validation_data = (X_valid, y_valid), epochs=30, batch_size=16, callbacks=[checkpoint, early_stopping])